# a) ineær regresjon i 3 dimensjoner:
<!-- Based on [linear-2d](https://gitlab.com/ntnu-tdat3025/regression/linear-2d) by Ole Christian Eidheim. -->

In [261]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, art3d
%matplotlib qt

## Import data from csv

In [262]:
train = pd.read_csv('day_length_weight.csv')

day = train.iloc[0::,0]
day_list = day.values.tolist()
x1_train = torch.tensor(day_list).reshape(-1, 1)

length = train.iloc[0::,1]
length_list = length.values.tolist()
x2_train = torch.tensor(length_list).reshape(-1, 1)

weight = train.iloc[0::,2]
weight_list = weight.values.tolist()
y_train = torch.tensor(weight_list).reshape(-1, 1)


# print(torch.cat((x1_train, x2_train),1))

# print(f"{x1_train[0:10] = }",f"\n{x2_train[0:10] = }",f"\n{y_train[0:10] = }")

# test data
# x_train = np.array([[1, 4.5], [1.5, 2], [2, 1], [3, 3.5], [4, 3], [5, 1], [6, 2]])
# y_train = np.array([[5], [3.5], [3], [4], [3], [1.5], [2]])

x_train = torch.cat((x1_train, x2_train),1).float()
# # .reshape(-2, 1)
# y_train = torch.tensor(y_train).reshape(-1, 1).float()

print(f"{x_train[0:10] = }",f"\n{y_train[0:10] = }")

x_train[0:10] = tensor([[ 604.0000,   84.8655],
        [ 964.0000,   92.8221],
        [1640.0000,   98.5218],
        [ 642.0000,   85.6332],
        [ 467.0000,   75.2944],
        [ 447.0000,   79.7825],
        [1450.0000,   99.4479],
        [1472.0000,  105.3180],
        [1661.0000,  116.2802],
        [1776.0000,  107.2823]]) 
y_train[0:10] = tensor([[13.0302],
        [14.7191],
        [21.0195],
        [12.7910],
        [10.5395],
        [ 9.3395],
        [12.4751],
        [12.1663],
        [20.8239],
        [16.3704]])


## Define Linear Regression Model class

In [263]:
# W_init = np.array([[-0.2], [0.53]])
# b_init = np.array([[3.1]])

class LinearRegressionModel:
    def __init__(self):
        # Model variables
        self.W = torch.tensor([[-0.2], [0.53]], requires_grad=True)  # requires_grad enables calculation of gradients
        self.b = torch.tensor([[3.1]], requires_grad=True)

    # Predictor
    def f(self, x):
        return x @ self.W + self.b  # @ corresponds to matrix multiplication

    # Uses Mean Squared Error
    def loss(self, x, y):
        return torch.nn.functional.mse_loss(self.f(x), y)

## Optimize with stochastic gradient descent (SGD)
PyTorch's built in optimizer does a lot of the work for us.

In [280]:
model = LinearRegressionModel()

# set variables
scale = 10**(-2)
epoch_count = 1000
step_length = 0.005

# downscale
x_train = x_train * scale
y_train = y_train * scale

# print(f"{model.W = }",f"\n{model.b = }")

# print(f"{x_train[0:6] = }",f"\n{y_train[0:6] = }")

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.SGD([model.W, model.b], step_length)

for epoch in range(epoch_count):
    
    if epoch % 100 == 0:
        print("W = %s, b = %s, loss = %s" % ([model.W[0].item(), model.W[1].item()] ,model.b.item(), model.loss(x_train, y_train).item()))

    model.loss(x_train, y_train).backward()  # Compute loss gradients

    optimizer.step()  # Perform optimization by adjusting W and b,

    optimizer.zero_grad()  # Clear gradients for next step


# correct for downscaling
x_train = x_train / scale
y_train = y_train / scale

W = [-0.20000000298023224, 0.5299999713897705], b = 3.0999999046325684, loss = 3.636127233505249
W = [-0.18743057548999786, 0.06492315977811813], b = 2.3848190307617188, loss = 1.405150055885315
W = [-0.12138956785202026, -0.25584959983825684], b = 1.8881454467773438, loss = 0.6910840272903442
W = [-0.0751366838812828, -0.48008808493614197], b = 1.5398821830749512, loss = 0.340629905462265
W = [-0.04274647310376167, -0.636699378490448], b = 1.2955888509750366, loss = 0.16862957179546356
W = [-0.020067891106009483, -0.7459343075752258], b = 1.1241331100463867, loss = 0.08421123027801514
W = [-0.004192830994725227, -0.8219802975654602], b = 1.0037047863006592, loss = 0.04277646541595459
W = [0.006915977690368891, -0.8747759461402893], b = 0.9190245866775513, loss = 0.0224373210221529
W = [0.014685766771435738, -0.9112841486930847], b = 0.8593884110450745, loss = 0.012451507151126862
W = [0.0201164148747921, -0.9363828897476196], b = 0.8172973990440369, loss = 0.007546922191977501


## Visualize data

In [277]:

# Print model variables and loss
print("W = %s, b = %s, loss = %s" % (model.W, model.b/scale, model.loss(x_train, y_train)))



# Create the figure
fig = plt.figure()

# Add an axes
ax = fig.add_subplot(111,projection='3d')

# xx, yy = np.meshgrid([0,1], [0, 1])
# z = (9 - xx - yy) / 2 

# # plot the plane
# ax.plot_surface(xx, yy, z, alpha=0.5)


# x1_grid, x2_grid = np.meshgrid(np.linspace(1, 6, 10), np.linspace(1, 4.5, 10))
# y_grid = np.empty([10, 10])

# ax.plot_wireframe(x1_grid, x2_grid, y_grid, color='green')


# print("torch max x_train",torch.max(x_train, 0).values, "---")

x = torch.stack([torch.min(x_train, 0).values, torch.max(x_train, 0).values])

print(f"{x = }")

print(x[:,0].reshape(-1,1))
print(x[:,1].reshape(-1,1))
print(model.f(x).detach())
# print(f"{[x[:,0].reshape(-1,1),x[:,1], model.f(x).detach()] = }")

# x[:,0].reshape(-1,1),x[:,1].reshape(-1,1), model.f(x).detach()

# plt.plot([0,1852],[0,118],[model.b,0], label='$\\hat y = f(x) = xW+b$')

def display(model):

    x1_grid, x2_grid = np.meshgrid(np.linspace(x[0,0], x[1,0], 10), np.linspace(x[0,1], x[1,1], 10))
    y_grid = np.empty([10, 10])
    for i in range(0, x1_grid.shape[0]):
        for j in range(0, x1_grid.shape[1]):
            x_val = torch.tensor( [[ x1_grid[i, j], x2_grid[i, j] ]] ).float()
            y_grid[i, j] = model.f(x_val)

    ax.plot_wireframe(x1_grid, x2_grid, y_grid, color='orange')

    # and plot the point 
    ax.scatter(x_train[:,0],x_train[:,1],y_train,  color='blue')

    plt.draw()
    plt.pause(0.02)
    ax.cla()




# plt.show()

W = tensor([[ 0.0293],
        [-0.8628]], requires_grad=True), b = tensor([[63.2279]], grad_fn=<DivBackward0>), loss = tensor(3929.2141, grad_fn=<MseLossBackward>)
x = tensor([[   2.0000,   49.4548],
        [1852.9999,  118.5752]])
tensor([[   2.0000],
        [1852.9999]])
tensor([[ 49.4548],
        [118.5752]])
tensor([[-41.9770],
        [-47.3003]])


In [281]:
a = torch.tensor([[ 604,84 ]]).float()
print()
print("f(604, 84) =" , model.f(a).item())


f(604, 84) = -64.8649673461914
